In [1]:
import pandas as pd
import altair as alt
import os 
from vega_datasets import data

In [2]:
cuisine_mapping = pd.read_csv('data/Cuisinne Mapping.csv')

In [39]:
# List files in data/Taste/
files = os.listdir('data/Taste/')
# Read in all files with pandas
# Filename as a column
dfs = []
for file in files:
    dfs.append(pd.read_csv('data/Taste/' + file))
    
# Concatenate all files
taste = pd.concat(dfs)

In [46]:
taste = pd.read_csv('./taste.csv')
taste

,Unnamed: 0,id,cuisine,title,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness
0,0,473895,European,Rigatoni Brussel Sprout Bake + Figs,100.00,16.06,34.38,14.20,10.75,65.81,1800000.0
1,1,204988,European,Parting Glass Belgian Dubbel (For Intermediate...,47.48,100.00,27.89,26.62,37.55,54.74,0.0
2,2,758105,European,Late Summer Minestrone with Butternut Squash a...,80.16,91.51,94.27,100.00,85.02,83.32,0.0
3,3,603272,European,Greek Salad Quinoa Bowl,31.41,15.69,36.00,25.57,2.50,100.00,10000.0
4,4,491863,European,Green Goddess Pasta Primavera,70.37,34.30,100.00,25.42,23.32,58.79,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4165,495,328195,Cajun,Chicken Jambalaya II,54.60,100.00,35.61,27.20,49.16,66.34,79450.0
4166,496,542568,Cajun,Cajun Shrimp and Quinoa Casserole,22.83,100.00,13.39,16.57,70.14,64.23,0.0
4167,497,316384,Cajun,Vegetable Gumbo,31.68,100.00,17.23,29.38,52.58,65.06,5950.0
4168,498,34805,Cajun,Half-hour Chicken Gumbo,33.88,98.51,39.61,25.99,59.44,100.00,15100.0


In [47]:
# Merge with cuisine mapping
mapping = taste.merge(cuisine_mapping, on='cuisine')
# Filter out those with blank "ISO 31661 Numeric"
mapping_filtered = mapping[mapping['ISO 31661 Numeric'].notnull()]
mapping_filtered.loc[:,'id'] = mapping_filtered['ISO 31661 Numeric'].astype(int).astype(str)

/home/johnymah/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [51]:
mapping_filtered = mapping_filtered[['id', 'sweetness', 'saltiness', 'sourness', 'bitterness', 'savoriness', 'fattiness', 'spiciness']]
mapping_filtered.columns.to_list()

['id',
 'sweetness',
 'saltiness',
 'sourness',
 'bitterness',
 'savoriness',
 'fattiness',
 'spiciness']

In [49]:
source = alt.topo_feature(data.world_110m.url, 'countries')
sphere = alt.sphere()
graticule = alt.graticule()

# flavors = mapping_filtered.columns[0:7]
# input_dropdown = alt.binding_select(options=flavors, name='Flavor')
# selection = alt.selection_single(fields=['Origin'], bind=input_dropdown)

flavor = 'sweetness'

# Layering and configuring the components
alt.Chart(source).mark_geoshape(
    stroke='black'
    ).encode(
        color= flavor + ':Q'
    ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(mapping_filtered, 'id', [flavor])
).project(
    'naturalEarth1'
).properties(width=600, height=400).configure_view(stroke=None)

alt.Chart(...)

In [43]:
# Same as above, but with a filter for the flavor

# Keep first 8 columns of mapping_filtered
mapping_filtered_flavor = mapping_filtered[mapping_filtered.columns[0:8]]
mapping_filtered_flavor

,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness,id
170,100.00,44.25,97.58,38.63,22.14,44.58,8200.0,840
171,100.00,48.26,10.31,2.98,29.82,53.99,0.0,840
172,8.56,3.64,0.00,0.00,0.00,100.00,160000.0,840
173,98.80,60.79,10.53,27.40,33.21,100.00,0.0,840
174,100.00,3.65,0.00,0.00,0.00,43.50,0.0,840
...,...,...,...,...,...,...,...,...
11040,100.00,28.59,29.04,41.64,49.62,36.24,375000.0,704
11041,100.00,26.38,48.94,26.21,10.80,55.32,375000.0,704
11042,25.99,100.00,11.19,14.30,65.68,85.20,0.0,704
11043,22.20,100.00,15.77,22.69,69.06,61.42,0.0,704


In [23]:


mapping_filtered_flavor_pivoted = pd.melt(mapping_filtered_flavor, id_vars=['id'], value_vars=mapping_filtered_flavor.columns[1:8])
# Melt with 'id' as key

In [9]:
# Take average of value by id, variabl
averaged = mapping_filtered_flavor_pivoted.groupby(['id','variable']).mean()
# Reset hierarchical index
averaged = averaged.reset_index()

In [10]:
averaged

,id,variable,value
0,156,bitterness,3.345268e+01
1,156,fattiness,6.659982e+01
2,156,saltiness,8.688942e+01
3,156,savoriness,5.350402e+01
4,156,sourness,2.619758e+01
...,...,...,...
73,840,fattiness,6.700252e+01
74,840,saltiness,7.291652e+01
75,840,savoriness,4.397934e+01
76,840,sourness,4.188214e+01


In [14]:
## NOT WORKING - needs debugged (only showing spiciness for some reason)
input_dropdown = alt.binding_select(options=list(mapping_filtered_flavor.columns[0:7]), name='Flavor')
selection = alt.selection_single(fields=['variable'], bind=input_dropdown)

# Create same map as above but with a filter for the flavor
# Layering and configuring the components
chart = alt.Chart(source).mark_geoshape(
    ).encode(
        color= 'value:Q',
        # opacity = 'variable:N'
    ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(averaged, 'id', ['value','variable'])
).project(
    'equirectangular'
).properties(width=600, height=400
).add_selection(
    selection
).transform_filter(selection).facet(
    facet='variable:N',
    columns=2
)
chart

alt.FacetChart(...)

In [27]:
all_flavors = list(mapping_filtered_flavor.columns[0:7])
all_flavors

['sweetness',
 'saltiness',
 'sourness',
 'bitterness',
 'savoriness',
 'fattiness',
 'spiciness']

In [50]:
charts = []
for selected_flavor in all_flavors:
    # filter source where variable == selected_flavor
    selected_averaged = averaged[averaged['variable'] == selected_flavor]

    chart = alt.Chart(source).mark_geoshape(
        ).encode(
            color= 'value:Q',
            # opacity = 'variable:N'
        ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(selected_averaged, 'id', ['value','variable'])
    ).project(
        'equirectangular'
    ).properties(width=600, height=400, title= selected_flavor
    )
    charts.append(chart)
# Display a grid of charts
alt.vconcat((charts[1] | charts[2]),
(charts[3]| charts[4]),
(charts[5]| charts[6]),
).resolve_scale(
    color='independent'
)

AttributeError: 'Chart' object has no attribute 'tooltip'

In [51]:
chart = alt.vconcat((charts[1] | charts[2]).resolve_scale(color='independent'),
(charts[3]| charts[4]).resolve_scale(color='independent'),
(charts[5]| charts[6]).resolve_scale(color='independent'),
).resolve_scale(
    color='independent'
)
chart

IndexError: list index out of range

In [49]:
print(chart.to_json(indent=2))

{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.17.0.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 400
    }
  },
  "data": {
    "format": {
      "feature": "countries",
      "type": "topojson"
    },
    "url": "https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json"
  },
  "datasets": {
    "data-2b4aa875a2ec0b08a2c41982e31425f7": [
      {
        "id": "156",
        "value": 804556.0192999999,
        "variable": "spiciness"
      },
      {
        "id": "250",
        "value": 155906.4766,
        "variable": "spiciness"
      },
      {
        "id": "276",
        "value": 200386.26668,
        "variable": "spiciness"
      },
      {
        "id": "356",
        "value": 1105402.59668,
        "variable": "spiciness"
      },
      {
        "id": "372",
        "value": 177171.12253999998,
        "variable": "spiciness"
      },
      {
        "id": "380",
        "value": 80279.51329999999,
      